In [2]:
import os
import time

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import TensorBoard
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

MODEL_NAME = "MegaClassifier_a"
VERSION_BASE = "v2"
EPOCHS = 10
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

DATASET_CSV = os.path.abspath("./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv")
DATASET_PATH = os.path.dirname(DATASET_CSV)

dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset['file_name'] = dataset['file_name'].apply(lambda x: os.path.join(DATASET_PATH, x))
dataset['binary_label'] = dataset['binary_label'].astype(str)

train_df = dataset[dataset['subset'] == 'train']
validation_df = dataset[dataset['subset'] == 'validation']
test_df = dataset[dataset['subset'] == 'test']

class_weights = compute_class_weight(class_weight="balanced",
                                     classes=np.unique(train_df['binary_label']),
                                     y=train_df['binary_label'])
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}


def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -K.mean(alpha * K.pow(1. - pt, gamma) * K.log(pt))
        return loss

    return loss


def train_and_evaluate(loss_function, loss_name, class_weights=None, VERSION_INDEX=0):
    print(f"\n🔹 Entrenando con función de pérdida: {loss_name} 🔹")

    # Generadores de imágenes
    train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validation_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
    )

    mobilenet_v2 = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    mobilenet_v2.trainable = False

    model = tf.keras.Sequential([
        mobilenet_v2,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ], name=f"{MODEL_NAME}_{VERSION_BASE}.{VERSION_INDEX}")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=loss_function,
        metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")],
    )

    start_time = time.time()
    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[
            TensorBoard(log_dir=f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}"),
        ],
        class_weight=class_weights  # Solo aplicable en Weighted Binary Crossentropy
    )
    training_time = time.time() - start_time

    results = model.evaluate(test_images)

    history_df = pd.DataFrame(history.history)
    os.makedirs(f"./logs/{MODEL_NAME}/{VERSION_BASE}/{loss_name}", exist_ok=True)
    history_df.to_csv(
        f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/history_{VERSION_BASE}.{VERSION_INDEX}.csv",
        index=False)

    metric_names = history.model.metrics_names
    evaluation_results = {("test_" + name): value for name, value in zip(metric_names, results)}
    evaluation_results["loss_function"] = loss_name
    evaluation_results["training_time"] = training_time

    results_df = pd.DataFrame([evaluation_results])
    results_df.to_csv(
        f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/results_{VERSION_BASE}.{VERSION_INDEX}.csv",
        index=False)

    print(f"\n📉 Loss: {results[0]:.4f}")
    print(f"🎯 Accuracy: {results[1]:.4%}")
    print(f"✅ Precision: {results[2]:.4%}")
    print(f"🔄 Recall: {results[3]:.4%}")
    print(f"📊 AUC: {results[4]:.4f}")
    print(f"⏳ Tiempo de entrenamiento: {training_time:.2f} segundos")

    return results_df


count = 0
all_results = []
loss_functions = {
    "BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy(),
    "FocalLoss": focal_loss(alpha=0.25, gamma=2.0),
    "WeightedBinaryCrossentropy": tf.keras.losses.BinaryCrossentropy()
}

for loss_name, loss_function in loss_functions.items():
    class_weights_to_use = class_weight_dict if loss_name == "WeightedBinaryCrossentropy" else None
    results_df = train_and_evaluate(loss_function, loss_name, class_weights=class_weights_to_use, VERSION_INDEX=count)
    all_results.append(results_df)
    count += 1

final_results = pd.concat(all_results, ignore_index=True)
final_results.to_csv(f"./logs/{MODEL_NAME}/{VERSION_BASE}/loss_comparison_results.csv", index=False)

print("\n✅ ¡Experimento con diferentes funciones de pérdida completado!")


🔹 Entrenando con función de pérdida: BinaryCrossentropy 🔹
Found 17054 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.


2025-02-23 19:08:53.080072: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-02-23 19:08:53.080097: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-23 19:08:53.080102: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-23 19:08:53.080151: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-23 19:08:53.080421: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2025-02-23 19:08:54.765409: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - ETA: 0s - loss: 0.2393 - accuracy: 0.8949 - precision: 0.9169 - recall: 0.9505 - auc: 0.9486

...

Epoch 1/10


2025-02-23 19:27:15.876727: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - ETA: 0s - loss: 0.3013 - accuracy: 0.8562 - precision: 0.9636 - recall: 0.8465 - auc: 0.9438

2025-02-23 19:27:59.639547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - 56s 103ms/step - loss: 0.3013 - accuracy: 0.8562 - precision: 0.9636 - recall: 0.8465 - auc: 0.9438 - val_loss: 0.2241 - val_accuracy: 0.9011 - val_precision: 0.9767 - val_recall: 0.8713 - val_auc: 0.9769
Epoch 2/10
533/533 [==============================] - 55s 103ms/step - loss: 0.2269 - accuracy: 0.8952 - precision: 0.9772 - recall: 0.8855 - auc: 0.9675 - val_loss: 0.2116 - val_accuracy: 0.9088 - val_precision: 0.9804 - val_recall: 0.8798 - val_auc: 0.9785
Epoch 3/10
533/533 [==============================] - 56s 106ms/step - loss: 0.2082 - accuracy: 0.9064 - precision: 0.9805 - recall: 0.8971 - auc: 0.9722 - val_loss: 0.1762 - val_accuracy: 0.9293 - val_precision: 0.9689 - val_recall: 0.9228 - val_auc: 0.9811
Epoch 4/10
533/533 [==============================] - 56s 104ms/step - loss: 0.1978 - accuracy: 0.9113 - precision: 0.9813 - recall: 0.9027 - auc: 0.9748 - val_loss: 0.1930 - val_accuracy: 0.9186 - val_precision: 0.9777 - val_recall: 